In [145]:
!pip install tensorflowjs

^C


In [1]:
import cv2
import mediapipe as mp
import tensorflow as tf
import csv
import numpy as np
import os
import pandas as pd
import csv
import itertools
import sys
import tempfile
import tqdm

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [43]:
# Download model from TF Hub and check out inference code from GitHub
# movenet_thunder.tflite = wget.download('https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite')
# # !wget -q -O movenet_thunder.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
# !git clone https://github.com/tensorflow/examples.git
# pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples/lite/examples/pose_estimation/raspberry_pi')
# sys.path.append(pose_sample_rpi_path)

# Load MoveNet Thunder model
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.

  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.

  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape

  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)

  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(), 
                            reset_crop_region=False)

  return person

In [68]:
def detect(input_tensor, inference_count=3):
    movenet.detect(input_tensor.numpy(), reset_crop_region=True)

    for _ in range(inference_count - 1):
        detection = movenet.detect(input_tensor.numpy(),
                                   reset_crop_region=False)

    return detection

In [72]:
class Preprocessor(object):
    #     this class preprocess pose samples, it predicts keypoints on the images
    #     and save those keypoints in a csv file for the later use in the classification task

    def __init__(self, images_in_folder,
                 csvs_out_path):
        self._images_in_folder = images_in_folder
        self._csvs_out_path = csvs_out_path
        self._csvs_out_folder_per_class = 'csv_per_pose'
        self._message = []

        if(self._csvs_out_folder_per_class not in os.listdir()):
            os.makedirs(self._csvs_out_folder_per_class)

#             get list of pose classes
        self._pose_class_names = sorted(
            [n for n in os.listdir(images_in_folder)]
        )

    def process(self, detection_threshold=0.1):
        #             Preprocess the images in the given folder
        for pose_class_name in self._pose_class_names:
            #                 paths for pose class
            images_in_folder = os.path.join(
                self._images_in_folder, pose_class_name)
            csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                        pose_class_name + '.csv'
                                        )
#               Detect landmarks in each images and write it to the csv files
            with open(csv_out_path, 'w') as csv_out_file:
                csv_out_writer = csv.writer(csv_out_file,
                                            delimiter=',',
                                            quoting=csv.QUOTE_MINIMAL
                                            )
    #             get the list of images
                image_names = sorted(
                    [n for n in os.listdir(images_in_folder)]
                )
                valid_image_count = 0
                # Detect pose landmarks in each image
                for image_name in tqdm.tqdm(image_names):
                    image_path = os.path.join(images_in_folder, image_name)

                    try:
                        image = tf.io.read_file(image_path)
                        image = tf.io.decode_jpeg(image)
                    except:
                        self._message.append(
                            'Skipped' + image_path + ' Invalid image')
                        continue

                    # skip images that is not RGB
                    if image.shape[2] != 3:
                        self.message.append(
                            'Skipped' + image_path + ' Image is not in RGB')
                        continue

                    person = detect(image)

                    # Save landmarks if all landmarks above than the threshold
                    min_landmark_score = min(
                        [keypoint.score for keypoint in person.keypoints])
                    should_keep_image = min_landmark_score >= detection_threshold
                    if not should_keep_image:
                        self._message.append(
                            'Skipped' + image_path + 'Keypoints score are below than threshold')
                        continue

                    valid_image_count += 1

                    # Get landmarks and scale it to the same size as the input image
                    pose_landmarks = np.array(
                        [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                         for keypoint in person.keypoints],
                        dtype=np.float32)

                    # writing the landmark coordinates to its csv files
                    coord = pose_landmarks.flatten().astype(np.str_).tolist()
                    csv_out_writer.writerow([image_name] + coord)

        print(self._message)

        # combine all per-csv class CSVs into a sigle csv file
        all_landmarks_df = self.all_landmarks_as_dataframe()
        all_landmarks_df.to_csv(self._csvs_out_path, index=False)

    def class_names(self):
        return self.pose_class_names

    def all_landmarks_as_dataframe(self):
        # Merging all csv for each class into a single csv file
        total_df = None
        for class_index, class_name in enumerate(self._pose_class_names):
            csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                        class_name + '.csv'
                                        )
            per_class_df = pd.read_csv(csv_out_path, header=None)

            # Add the labels
            per_class_df['class_no'] = [class_index]*len(per_class_df)
            per_class_df['class_name'] = [class_name]*len(per_class_df)

            # Append the folder name to the filename first column
            per_class_df[per_class_df.columns[0]] = class_name + \
                '/' + per_class_df[per_class_df.columns[0]]

            if total_df is None:
                total_df = per_class_df
            else:
                total_df = pd.concat([total_df, per_class_df], axis=0)

        list_name = [[bodypart.name + '_x', bodypart.name + '_y',
                      bodypart.name + '_score'] for bodypart in BodyPart]

        header_name = []
        for columns_name in list_name:
            header_name += columns_name
        header_name = ['filename'] + header_name
        header_map = {total_df.columns[i]: header_name[i]
                      for i in range(len(header_name))
                      }

        total_df.rename(header_map, axis=1, inplace=True)

        return total_df

In [73]:
images_in_folder = os.path.join('yoga_poses', 'train')
csvs_out_path = 'train_data.csv'
train_preprocessor = Preprocessor(
    images_in_folder,
    csvs_out_path
)
train_preprocessor.process()

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [03:05<00:00,  2.16it/s]


['Skippedyoga_poses\\train\\chair\\girl3_chair087_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair088_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair089_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair091.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair091_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair092.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair092_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair093.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair093_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\train\\chair\\girl3_chair094.jpgKeypoints score are below than threshold', '

In [74]:
images_in_folder = os.path.join('yoga_poses', 'test')
csvs_out_path = 'test_data.csv'
test_preprocessor = Preprocessor(
    images_in_folder,
    csvs_out_path
)
test_preprocessor.process()

100%|████████████████████████████████████████████████████████████████████████████████| 218/218 [01:43<00:00,  2.11it/s]


['Skippedyoga_poses\\test\\cobra\\guy3_cobra036_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra048.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra050.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra051.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra051_flipped.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra052.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra053.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra054.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra055.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra056.jpgKeypoints score are below than threshold', 'Skippedyoga_poses\\test\\cobra\\guy3_cobra057.jpgKey

In [24]:
# # Download the preprocessed CSV files which are the same as the output of step 1
# # filename = 'train_data.csv'
# train_data = wget.download('http://download.tensorflow.org/data/pose_classification/yoga_train_data.csv')
# test_data = wget.download('http://download.tensorflow.org/data/pose_classification/yoga_test_data.csv')
# # !wget -O train_data.csv http://download.tensorflow.org/data/pose_classification/yoga_train_data.csv
# # !wget -O test_data.csv http://download.tensorflow.org/data/pose_classification/yoga_test_data.csv

# csvs_out_train_path = 'train_data.csv'
# csvs_out_test_path = 'test_data.csv'
# is_skipped_step_1 = True

100% [............................................................................] 318187 / 318187

In [128]:
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')

In [148]:
train_df['class_name'].unique()

array(['chair', 'cobra', 'dog', 'no_pose', 'shoudler_stand', 'traingle',
       'tree', 'warrior'], dtype=object)

In [80]:
def load_pose_landmarks(csv_path):
    dataframe = pd.read_csv(csv_path)
    df_to_process = dataframe.copy()

  # Drop the file_name columns as you don't need it during training.
    

  # Extract the list of class names
    classes = df_to_process['class_name'].unique()

  # Extract the labels
    y = df_to_process['class_no']
    
    df_to_process.drop(columns=['filename','class_name','class_no'],axis=1, inplace=True)

  # Convert the input features and labels into the correct format for training.
    X = df_to_process.astype('float64')
    y = keras.utils.to_categorical(y)

    return X, y, classes, dataframe

In [131]:
x,y,classes ,df = load_pose_landmarks('train_data.csv')

In [132]:
x

,NOSE_x,NOSE_y,NOSE_score,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_score,RIGHT_EYE_x,RIGHT_EYE_y,RIGHT_EYE_score,LEFT_EAR_x,...,LEFT_KNEE_score,RIGHT_KNEE_x,RIGHT_KNEE_y,RIGHT_KNEE_score,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_score,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_score
0,158.0,91.0,0.486214,159.0,86.0,0.567006,154.0,86.0,0.471731,155.0,...,0.652460,166.0,223.0,0.659201,152.0,259.0,0.501736,148.0,262.0,0.536224
1,138.0,91.0,0.602957,143.0,87.0,0.507938,139.0,87.0,0.732789,155.0,...,0.859733,125.0,220.0,0.879398,152.0,263.0,0.531063,147.0,259.0,0.663975
2,165.0,100.0,0.674502,165.0,95.0,0.771247,161.0,95.0,0.767397,160.0,...,0.841010,173.0,226.0,0.892319,155.0,262.0,0.701863,152.0,263.0,0.707526
3,134.0,101.0,0.498700,138.0,96.0,0.598085,134.0,95.0,0.545088,151.0,...,0.705889,120.0,224.0,0.837559,150.0,265.0,0.762652,145.0,261.0,0.616945
4,165.0,102.0,0.640303,166.0,99.0,0.595475,162.0,98.0,0.702522,160.0,...,0.780941,173.0,228.0,0.828626,156.0,263.0,0.630900,152.0,264.0,0.672576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,183.0,196.0,0.514371,187.0,190.0,0.481944,178.0,191.0,0.552120,193.0,...,0.423765,141.0,179.0,0.197730,127.0,106.0,0.345543,127.0,106.0,0.509772
1507,120.0,195.0,0.396956,124.0,191.0,0.564491,114.0,191.0,0.470278,134.0,...,0.804588,156.0,130.0,0.362384,172.0,105.0,0.810996,172.0,104.0,0.381399
1508,128.0,186.0,0.231476,132.0,184.0,0.505300,122.0,182.0,0.578729,144.0,...,0.217305,141.0,202.0,0.400086,165.0,194.0,0.184848,142.0,226.0,0.451131
1509,160.0,117.0,0.552652,166.0,111.0,0.740929,155.0,112.0,0.756139,174.0,...,0.249607,163.0,201.0,0.796670,194.0,127.0,0.101683,192.0,126.0,0.113600


In [133]:
def drop_score(df):
    drop_col=[]
    for col in df.columns:
        s = col.split('_')
        if s[len(s)-1]=='score':
            drop_col.append(col)
    return df.drop(drop_col,axis=1)

In [134]:
x = drop_score(x)

In [136]:
x

,NOSE_x,NOSE_y,LEFT_EYE_x,LEFT_EYE_y,RIGHT_EYE_x,RIGHT_EYE_y,LEFT_EAR_x,LEFT_EAR_y,RIGHT_EAR_x,RIGHT_EAR_y,...,RIGHT_HIP_x,RIGHT_HIP_y,LEFT_KNEE_x,LEFT_KNEE_y,RIGHT_KNEE_x,RIGHT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y
0,158.0,91.0,159.0,86.0,154.0,86.0,155.0,87.0,144.0,87.0,...,142.0,172.0,174.0,220.0,166.0,223.0,152.0,259.0,148.0,262.0
1,138.0,91.0,143.0,87.0,139.0,87.0,155.0,88.0,147.0,87.0,...,144.0,171.0,134.0,222.0,125.0,220.0,152.0,263.0,147.0,259.0
2,165.0,100.0,165.0,95.0,161.0,95.0,160.0,97.0,150.0,97.0,...,139.0,179.0,180.0,222.0,173.0,226.0,155.0,262.0,152.0,263.0
3,134.0,101.0,138.0,96.0,134.0,95.0,151.0,97.0,143.0,97.0,...,145.0,178.0,127.0,226.0,120.0,224.0,150.0,265.0,145.0,261.0
4,165.0,102.0,166.0,99.0,162.0,98.0,160.0,100.0,151.0,99.0,...,139.0,182.0,181.0,224.0,173.0,228.0,156.0,263.0,152.0,264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,183.0,196.0,187.0,190.0,178.0,191.0,193.0,175.0,166.0,177.0,...,139.0,151.0,136.0,127.0,141.0,179.0,127.0,106.0,127.0,106.0
1507,120.0,195.0,124.0,191.0,114.0,191.0,134.0,177.0,107.0,177.0,...,135.0,141.0,166.0,128.0,156.0,130.0,172.0,105.0,172.0,104.0
1508,128.0,186.0,132.0,184.0,122.0,182.0,144.0,169.0,114.0,170.0,...,134.0,162.0,159.0,189.0,141.0,202.0,165.0,194.0,142.0,226.0
1509,160.0,117.0,166.0,111.0,155.0,112.0,174.0,96.0,148.0,97.0,...,155.0,144.0,191.0,139.0,163.0,201.0,194.0,127.0,192.0,126.0


In [102]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [117]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

In [118]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=x.shape[1]))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               8960      
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 8)                 520       
                                                                 
Total params: 50,632
Trainable params: 50,632
Non-trai

In [119]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=20, min_delta =0.1 ,restore_best_weights=True)

In [146]:
history = model.fit(x.values, y, batch_size=25,epochs=1000,callbacks=[callback])

Epoch 1/1000
61/61 [==============================] - 1s 4ms/step - loss: 0.2492 - accuracy: 0.9186
Epoch 2/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.3383 - accuracy: 0.8848
Epoch 3/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.3535 - accuracy: 0.8875
Epoch 4/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.2891 - accuracy: 0.9047
Epoch 5/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.2776 - accuracy: 0.9060
Epoch 6/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.2948 - accuracy: 0.9073
Epoch 7/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.2167 - accuracy: 0.9292
Epoch 8/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.2288 - accuracy: 0.9219
Epoch 9/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.2280 - accuracy: 0.9338
Epoch 10/1000
61/61 [==============================] - 0s 4ms/step - loss: 0.3015 - accuracy: 0.8994

In [138]:
x_test,y_test,classes ,df_test = load_pose_landmarks('test_data.csv')

In [142]:
x_test = drop_score(x_test)

In [147]:
model.evaluate(x_test,y_test)

28/28 [==============================] - 0s 3ms/step - loss: 0.0549 - accuracy: 0.9886


[0.05485977604985237, 0.9885975122451782]

In [149]:
model.predict(x_test)

28/28 [==============================] - 1s 3ms/step


array([[9.99922156e-01, 1.19728572e-09, 4.97678954e-10, ...,
        1.62676741e-07, 7.76045927e-05, 1.01414159e-07],
       [9.99929428e-01, 1.79729354e-09, 8.82257212e-10, ...,
        1.86916594e-07, 7.02411198e-05, 1.23400440e-07],
       [9.99950051e-01, 1.55430935e-09, 5.90100080e-10, ...,
        1.41730709e-07, 4.96841458e-05, 1.07865404e-07],
       ...,
       [2.55643646e-03, 1.42225787e-01, 3.16357473e-05, ...,
        1.39237067e-03, 4.89899976e-05, 8.53731275e-01],
       [3.32100987e-02, 1.22870217e-04, 7.85050273e-04, ...,
        3.30909435e-03, 2.16394570e-03, 9.59850073e-01],
       [8.25211313e-03, 1.55155182e-01, 2.68651893e-05, ...,
        2.83804256e-03, 1.81207710e-04, 8.33517730e-01]], dtype=float32)